In [1]:
# STEP 0: Must be done before anything each time notebook is loaded
# Import PySplit
import pysplit as py
import numpy as np
import pickle as pickle
from pprint import pprint
import os

In [20]:
# INITIALISE TRAJECTORY GROUP
# Group trajectories and then cycle through a group to determine distance, vector and rain status.
# check out traj_trajgroup_basics_examle.py

# Initialise Trajectories and Create a Trajectory Group
trajg=[]
trajg = py.make_trajectorygroup(r'C:/hysplit4/pysplit/sumatra_ic_test3/*1300*')

#had to update hypath.py to latest version from pysplit (comment available in .py code)
for traj in trajg:
    traj.calculate_distance()
    traj.calculate_vector()
    
print('Total trajectories in group: ',trajg.trajcount)

Total trajectories in group:  179


In [21]:
# test for initial height (850hPa)
# average P at timestep 0 in traj group 
p_i=[]
for traj in trajg:
    p_i.append(traj.data['Pressure'][0])

np.mean(p_i)

854.4290502793295

In [22]:
# integration error filtering
#first load reverse Traj
for traj in trajg:
    traj.load_reversetraj()
#then calculate integration error
# Values computed when calling ``Trajectory.calculate_integrationerr()``:
#     ``Trajectory.integration_error``, the relative error (%)
#     ``Trajectory.integration_error_abs``, the absolute error (meters)
for traj in trajg:
    traj.calculate_integrationerr()

C:\Users\jenni\Anaconda3\envs\pysplitenv\lib\site-packages\pysplit\hypath.py:187: RuntimeWarning: invalid value encountered in arccos
  np.cos(lon[0] - lon)) * 6371) * 1000


In [23]:
for traj in trajg:
    print(traj.integration_error)
    
# [traj.integration_error for traj in trajg]

0.002665424673300689
0.011680283760709618
0.0018109390268980746
0.002958353972853913
0.005469286713649229
0.0036995919591066503
0.7337370631849542
0.0033523823978149183
0.004239365146295249
0.007248181096920348
0.03237369226344971
0.005904185117240831
0.0047133596148104355
0.002616888066952887
0.007644547046994385
0.0035129602612340325
0.005495418782780535
0.002312875937081666
0.0102866027693863
0.02759237494792518
0.004404231244212474
0.3335213352074967
0.002570757168950301
0.003407772529966432
0.009013388003807424
0.0017350680006570905
0.06236315499367823
0.011313294511983854
0.0062550789564374066
0.011029175049035217
0.00902530462192924
0.0008634614447615678
0.016056400916119637
0.02128883278727512
0.008614567051588834
0.006492090948869334
0.003154346052878872
0.009051012562775736
0.0020345879103940197
0.07654566487157526
0.0013311664703971567
0.012454013276590022
0.0016827248503012903
0.004768410501117323
0.007238674956717007
0.021360931789834118
0.0024191911536847878
6.93746004564

In [24]:
# Once we can have these values, one action we can take is to discard the "bad"
# trajectories.  A reasonable way to define "bad" trajectories are those with
# integration errors greater than two standard deviations above the mean:

relative_errors = [traj.integration_error for traj in trajg]
cutoff = np.mean(relative_errors) + (np.std(relative_errors) * 2)

print('Integration error upper limit: ', cutoff)

Integration error upper limit:  7.366431955571946


In [25]:
# With this data, we can cycle through ``trajgroup`` and either identify "good"
# trajectories to put in a new ``TrajectoryGroup``
# (see ``traj_trajgroup_basics_example.py``), or, as below, identify "bad"
# trajectories to remove from ``trajgroup``.  In this example, we make a list of
# the identifiers (``Trajectory.trajid``) of "bad" trajectories, then pass the
# list to the ``TrajectoryGroup.pop()`` method, which removes the indicated
# trajectories from ``trajgroup``.

# ``TrajectoryGroup.pop()`` accepts a list of ``Trajectory.trajid``, a single 
# ``Trajectory.trajid``, or an index.  If none of the above are specified it
# defaults to the last ``Trajectory`` in ``TrajectoryGroup``.  As with
# ``list.pop()``, performing ``TrajectoryGroup.pop()``while iterating over
# ``TrajectoryGroup`` will lead to unexpected behavior. 

# ``TrajectoryGroup.pop()`` returns a ``Trajectory``, if one
# ``Trajectory.trajid`` or an index is given, or a ``TrajectoryGroup``,
# if given a list of ``Trajectory.trajid``.

bad = []
for traj in trajg:
    if traj.integration_error > cutoff:
        bad.append(traj.trajid)

print('Expectation: ', trajg.trajcount, 'trajectories -', len(bad),
      'bad trajectories =', trajg.trajcount-len(bad), 'trajectories')

trajg.pop(trajid=bad)

print('Result: ', trajg.trajcount, 'trajectories')

# for traj in trajgroup:
#     print(traj.integration_error)

Expectation:  179 trajectories - 8 bad trajectories = 171 trajectories
Result:  171 trajectories


In [ ]:
#list of rainy traj
rainylist = []

for traj in trajg:
    traj.set_rainstatus()
    if traj.rainy:
        rainylist.append(traj)
        
rainy_trajgroup = py.TrajectoryGroup(rainylist)
print('Total trajectories in group: ',rainy_trajgroup.trajcount)

In [ ]:
# moisture uptake
moisture = []
for traj in rainy_trajgroup:
    traj.moisture_uptake(precipitation=-0.2,evaporation=0.2,interval=3)
    moisture.append(traj)

In [ ]:
moisture[1].uptake

In [ ]:
list(trajg[5].data)

In [ ]:
trajg[5].data['Rainfall'][0]

In [ ]:
type(trajg[5].data)